In [1]:
import cv2
import numpy as np
import glob
import os

def process_videos(video_paths, output_folder):
    points = []
    
    def select_point(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))
            cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
            cv2.imshow('Select Point', frame)
    
    first_frames = []
    
    # Step 1: Extract first frames and collect points
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        ret, frame = cap.read()
        if ret:
            first_frames.append((frame, video_path))
            cv2.imshow('Select Point', frame)
            cv2.setMouseCallback('Select Point', select_point)
            cv2.waitKey(0)
        cap.release()
    
    cv2.destroyAllWindows()
    
    # Step 2: Calculate mean point
    if not points:
        print("No points were selected.")
        return
    
    mean_point = np.mean(points, axis=0).astype(int)
    print(f"Mean point: {mean_point}")
    
    # Step 3: Align videos
    os.makedirs(output_folder, exist_ok=True)
    
    for (frame, video_path), point in zip(first_frames, points):
        dx, dy = mean_point[0] - point[0], mean_point[1] - point[1]
        cap = cv2.VideoCapture(video_path)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        output_path = os.path.join(output_folder, os.path.basename(video_path))
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            translation_matrix = np.float32([[1, 0, dx], [0, 1, dy]])
            translated_frame = cv2.warpAffine(frame, translation_matrix, (width, height))
            out.write(translated_frame)
        
        cap.release()
        out.release()
    
    print(f"Aligned videos saved in '{output_folder}'.")

In [4]:
video_files = glob.glob("C:/Users/dhers/Desktop/prueba/*.mp4")  # Update path to your videos
output_dir = "C:/Users/dhers/Desktop/prueba/aligned_videos"
process_videos(video_files, output_dir)

Mean point: [188 117]
Aligned videos saved in 'C:/Users/dhers/Desktop/prueba/aligned_videos'.
